In [1]:
import os
import math
import pandas as pd
import numpy as np
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

import datetime
import dateparser

import hydrofunctions as hf

import fbprophet 
from fbprophet.diagnostics import cross_validation

# For inputting City information to get distance to sensors.
from  geopy.geocoders import Nominatim
import geopy.distance

# Date time conversion registration.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


In [62]:
## Reading in site data from 'Gage_sites.txt'
file = 'Gage_sites.txt'
sites = pd.read_csv(file, sep = '\t', header = 39, skiprows = [40], dtype = {"site_no" : "str"})

sites.drop([65, 71, 77, 117, 118, 138, 203, 225, 320, 330, 331, 338], inplace = True) ##Removing sites without data

site_no = list(sites["site_no"])
site_nm = list(sites["station_nm"])
site_loc = sites.filter(['site_no', 'station_nm', 'dec_lat_va', 'dec_long_va'])

In [3]:
dbname = 'usgs_stream_db'
username = 'cadeadams'

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))

if not database_exists(engine.url):
    create_database(engine.url)

In [60]:
con = None
con = psycopg2.connect(database = dbname, user = username)

sql_query = """
SELECT * FROM site_locations;
"""

query_results = pd.read_sql_query(sql_query,con)

location = [38.7511041, -105.521384]
date = "Sep 23, 2020"
t = dateparser.parse(date)

site_no = query_results["site_no"]
site_lat = query_results["dec_lat_va"]
site_long = query_results["dec_long_va"]

sites_coord = pd.DataFrame([site_no, site_lat, site_long])
sites_coord = sites_coord.T

distance = []
for i in range(len(sites_coord)) :
    distance.append(geopy.distance.distance(location, sites_coord.iloc[i,1:]).miles)

query_results["distance"] = distance
site_no = pd.DataFrame(site_no)
site_no["distance"] = distance
query_results = query_results.sort_values(by = ["distance"])
site_no = site_no.sort_values(by = ["distance"])

count = 0
loc_lat = list()
loc_lon = list()
flow = list()
flow_upper = list()
flow_lower = list()
good_site = list()
good_dist = list()

for i in range(len(site_no)) :
    sql_query_model = """
                      SELECT * FROM n"""+site_no['site_no'].iloc[i]+"""_forecast;
                      """
    query_results_model = pd.read_sql_query(sql_query_model,con)
    if (t == query_results_model['ds']).any() :
        temp = query_results_model.loc[query_results_model['ds'] == t]
        if (temp['yhat'].iloc[0] > 100 and temp['yhat'].iloc[0] < 400) :
            loc_lat.append(float(query_results[i:i+1]["dec_lat_va"]))
            loc_lon.append(float(query_results[i:i+1]["dec_long_va"]))
            good_site.append(query_results[i:i+1]["site_no"].iloc[0])
            good_dist.append(query_results[i:i+1]["distance"].iloc[0])
            flow.append(temp['yhat'].iloc[0])
            flow_upper.append(temp['yhat_upper'].iloc[0])
            flow_lower.append(temp['yhat_lower'].iloc[0])
            count = count + 1
            if count == 3 :
                break
        else :
            continue

print(flow_upper)
print(flow)
print(flow_lower)
#loc1_lat = float(query_results[0:1]["dec_lat_va"])
#loc1_lon = float(query_results[0:1]["dec_long_va"])
#location_1 = pd.DataFrame([loc1_lat, loc1_lon])
#print(query_results.head())
#print(site_no.head())

#site_data_from_sql = pd.read_sql_query(sql_query_dat,con)
#site_model_from_sql = pd.read_sql_query(sql_query_mod,con)

[230.41453120574732, 299.12043536370714, 423.05635695726676]
[174.5370202258796, 253.48735318298114, 309.8214353732134]
[129.77860381352696, 205.5372636417656, 154.6292389192572]


In [98]:
con = None
con = psycopg2.connect(database = dbname, user = username)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


mape_baseline = list()
for site in site_no[179:] :
    sql_query = """
                SELECT * FROM n"""+site+""";
                """
    site_data_from_sql = pd.read_sql_query(sql_query,con)

    df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, daily_seasonality=True, interval_width = 0.25)
    df_prophet.fit(site_data_from_sql)

    cv_results = cross_validation(df_prophet, initial = '1095 days', period = '180 days', horizon = '365 days')
    cv_results.to_sql("n"+str(site)+"_cv", engine, if_exists='replace')
    mape_baseline.append(mean_absolute_percentage_error(cv_results.y, cv_results.yhat))


ValueError: Less data than horizon after initial window. Make horizon or initial shorter.

In [101]:
cross_val_all = pd.DataFrame()

site_no = site_no[0:180]
site_no = pd.DataFrame(site_no)
site_no.drop([18, 83, 84, 85, 89, 92, 101, 103, 104, 
              136, 138, 146, 148, 149, 150, 151, 179]
             , inplace = True)

site_no = list(site_no)
cross_val_all["site_no"] = site_no

mape_baseline = list()
for site in site_no :
    sql_query = """
                SELECT * FROM n"""+site+"""_cv;
                """
    site_cv_from_sql = pd.read_sql_query(sql_query,con)
    mape_baseline.append(mean_absolute_percentage_error(site_cv_from_sql.y, site_cv_from_sql.yhat))

cross_val_all["mape_baseline"] = mape_baseline
print(cross_val_all.head())

KeyError: '[ 18  83  84  85  89  92 101 103 104 136 138 146 148 149 150 151 179] not found in axis'

In [ ]:
n06711040 (207 days only) 18
n07103784 (257 days only) 83
n07103791 (257 days only) 84
n07103792 (471 days only) 85
n07103965 (676 days only) 89
n07103987 (646 days only) 92
n07105780 (814 days only) 101
n07105815 (868 days only) 103
n07105820 (867 days only) 104
n09019000 (405 days only) 136
n09021000 (677 days only) 138
n09032050 (288 days only) 146
n09032200 (295 days only) 148
n09032300 (304 days only) 149
n09032990 (337 days only) 150
n09033010 (309 days only) 151
n09060799 (867 days only) 179


In [76]:
site_no.index('06711040')

18

In [36]:
df_forecast = df_prophet.make_future_dataframe(periods=365 * 1, freq='D')
df_forecast = df_prophet.predict(df_forecast)
df_prophet.plot(df_forecast)
plt.xlabel('Date')
plt.yscale('log')
plt.ylabel('Discharge (cfs)')
plt.savefig('test.png')
plt.close()

In [56]:
df_forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
3513,2020-09-25,-932.269063,173.453690,236.958924,-932.269063,-932.269063,1133.681856,1133.681856,1133.681856,0.390274,0.390274,0.390274,1133.291582,1133.291582,1133.291582,0.0,0.0,0.0,201.412793
3514,2020-09-26,-932.165895,168.918588,231.281007,-932.165895,-932.165895,1133.189392,1133.189392,1133.189392,-0.603464,-0.603464,-0.603464,1133.792857,1133.792857,1133.792857,0.0,0.0,0.0,201.023497
3515,2020-09-27,-932.062727,155.987371,229.940654,-932.062727,-932.062727,1133.518593,1133.518593,1133.518593,-1.308300,-1.308300,-1.308300,1134.826893,1134.826893,1134.826893,0.0,0.0,0.0,201.455867
3516,2020-09-28,-931.959559,163.611683,240.809111,-931.959559,-931.959559,1136.049050,1136.049050,1136.049050,-0.191631,-0.191631,-0.191631,1136.240681,1136.240681,1136.240681,0.0,0.0,0.0,204.089492
3517,2020-09-29,-931.856391,174.700698,245.140438,-931.856391,-931.856391,1138.132011,1138.132011,1138.132011,0.356313,0.356313,0.356313,1137.775698,1137.775698,1137.775698,0.0,0.0,0.0,206.275620


In [46]:
#type(pd.to_datetime(site_model_from_sql['ds']))
date = "Sep 23, 2020"
t = dateparser.parse(date)
df_forecast.loc[df_forecast['ds'] == t]

             ds      trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
7571 2020-09-23  80.014778   47.740213  108.163608    80.014778    80.014778   

      additive_terms  additive_terms_lower  additive_terms_upper     daily  \
7571        9.691244              9.691244              9.691244 -5.712846   

      ...    weekly  weekly_lower  weekly_upper     yearly  yearly_lower  \
7571  ... -5.572282     -5.572282     -5.572282  20.976372     20.976372   

      yearly_upper  multiplicative_terms  multiplicative_terms_lower  \
7571     20.976372                   0.0                         0.0   

      multiplicative_terms_upper       yhat  
7571                         0.0  89.706022  

[1 rows x 22 columns]


# Everything below here is for collection and placing into PostgreSQL database. FBProphet models are also performed over all data collected.

In [4]:
#site_no.index('394220106431500') ## Has water temp data. Need to extract...

site_loc.to_sql('site_locations', engine, if_exists='replace')

In [5]:
##
# Pulling in data using hydrofunctions and saving to PostgreSQL database.
##
start = '2000-01-01'
end = str(datetime.datetime.today().strftime('%Y-%m-%d')) #Gets today's date.

for site in site_no :
    herring = hf.NWIS(site, 'dv', start, end)
    herring.get_data()
    my_dict = herring.json()
    df = hf.extract_nwis_df(my_dict)
    df.rename(index=str, columns = {"USGS:"+site+":00060:00003" : "y", 
                                    "USGS:"+site+":00060:00003_qualifiers" : "flags"}, 
             inplace = True)
    df['ds'] = df.index[:]
#    df['ds'].str.split(pat = ' ', expand = True)
    df.to_sql("n"+str(site), engine, if_exists='replace')

In [17]:
##
# Modeling data using fbprophet and saving to PostgreSQL database.
##
con = None
con = psycopg2.connect(database = dbname, user = username)

for site in site_no :
    sql_query = """
    SELECT * FROM n"""+site+""";
    """
    site_data_from_sql = pd.read_sql_query(sql_query,con)
    df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, daily_seasonality=True, interval_width = 0.25)
    df_prophet.fit(site_data_from_sql)
    df_forecast = df_prophet.make_future_dataframe(periods=365 * 1, freq='D')
    df_forecast = df_prophet.predict(df_forecast)
    df_forecast.to_sql("n"+str(site)+"_forecast", engine, if_exists='replace')

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

In [16]:
df.plot()
plt.xlabel('Date')
plt.yscale('log')
plt.ylabel('Discharge (cfs)')
plt.title('Daily Mean Discharge for Roaring Fork River')
plt.savefig('RRFork1.png')
plt.close()

In [9]:
df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, daily_seasonality=True, interval_width = 0.25)
df_prophet.fit(df)

In [10]:
df_forecast = df_prophet.make_future_dataframe(periods=365 * 1, freq='D')
df_forecast = df_prophet.predict(df_forecast)

In [12]:
df_prophet.plot_components(df_forecast)
plt.savefig('RRFork1_Components.png')
plt.close()

In [18]:
site_model_from_sql.tail()

,index,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
7691,7691,2021-01-21,0.967861,-0.579543,1.447869,0.967861,0.967861,-0.477573,-0.477573,-0.477573,...,0.020013,0.020013,0.020013,-2.623681,-2.623681,-2.623681,0.0,0.0,0.0,0.490288
7692,7692,2021-01-22,0.968049,-0.603034,1.530899,0.968049,0.968049,-0.491970,-0.491970,-0.491970,...,0.016397,0.016397,0.016397,-2.634462,-2.634462,-2.634462,0.0,0.0,0.0,0.476079
7693,7693,2021-01-23,0.968238,-0.522904,1.572616,0.968238,0.968238,-0.525330,-0.525330,-0.525330,...,-0.008017,-0.008017,-0.008017,-2.643407,-2.643407,-2.643407,0.0,0.0,0.0,0.442908
7694,7694,2021-01-24,0.968426,-0.578047,1.476846,0.968426,0.968426,-0.533844,-0.533844,-0.533844,...,-0.009711,-0.009711,-0.009711,-2.650228,-2.650228,-2.650228,0.0,0.0,0.0,0.434582
7695,7695,2021-01-25,0.968615,-0.388971,1.673874,0.968615,0.968615,-0.530558,-0.530558,-0.530558,...,-0.001953,-0.001953,-0.001953,-2.654699,-2.654699,-2.654699,0.0,0.0,0.0,0.438057
